In [32]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import math
import sklearn
import sklearn.preprocessing as preprocess
from sklearn.impute import SimpleImputer
import scipy.stats as stats
from sklearn.ensemble import AdaBoostRegressor
import joblib

# %matplotlib notebook
#補缺失值=>onehotencoding=>normalize=>observation(remove outlier)

In [33]:
#preprocessing
# train_log = pd.read_csv("data/train_log.csv", low_memory=False)
#下載資料 進行初步的觀察
data = pd.read_csv("data/test.csv", low_memory=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5723 entries, 0 to 5722
Data columns (total 70 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         5723 non-null   int64  
 1   P_ID       5723 non-null   int64  
 2   Date       5723 non-null   object 
 3   LOG ID     5723 non-null   int64  
 4   Sub. ID    5723 non-null   object 
 5   1-1_D      5027 non-null   object 
 6   1-1_M      5027 non-null   object 
 7   1-1_cell   5118 non-null   float64
 8   1-1_value  5118 non-null   float64
 9   1-2_D      5118 non-null   object 
 10  1-2_M      5118 non-null   object 
 11  1-2_cell   5118 non-null   float64
 12  1-2_value  5118 non-null   float64
 13  1-3_D      5118 non-null   object 
 14  1-3_M      5118 non-null   object 
 15  1-3_cell   5118 non-null   float64
 16  1-3_value  5118 non-null   float64
 17  2-1_D      5693 non-null   object 
 18  2-1_M      5693 non-null   object 
 19  2-1_cell   5693 non-null   float64
 20  2-1_valu

In [34]:
#初步觀察之後發現
data.drop(columns='4-2_D', inplace=True)
data.drop(columns='4-2_M', inplace=True)
data.drop(columns='Date', inplace=True)
data.drop(columns='Sub. ID', inplace=True)

In [35]:
#來確認該砍的是不是都砍了
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5723 entries, 0 to 5722
Data columns (total 66 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         5723 non-null   int64  
 1   P_ID       5723 non-null   int64  
 2   LOG ID     5723 non-null   int64  
 3   1-1_D      5027 non-null   object 
 4   1-1_M      5027 non-null   object 
 5   1-1_cell   5118 non-null   float64
 6   1-1_value  5118 non-null   float64
 7   1-2_D      5118 non-null   object 
 8   1-2_M      5118 non-null   object 
 9   1-2_cell   5118 non-null   float64
 10  1-2_value  5118 non-null   float64
 11  1-3_D      5118 non-null   object 
 12  1-3_M      5118 non-null   object 
 13  1-3_cell   5118 non-null   float64
 14  1-3_value  5118 non-null   float64
 15  2-1_D      5693 non-null   object 
 16  2-1_M      5693 non-null   object 
 17  2-1_cell   5693 non-null   float64
 18  2-1_value  5693 non-null   float64
 19  2-2_D      5693 non-null   object 
 20  2-2_M   

In [36]:
#填補缺失值
test_cate = data.select_dtypes(include=['object', 'category'])
cate_type = pd.Series(data.select_dtypes(include=['object', 'category']).columns)

test_float = data.select_dtypes(include=['float', 'int'])
float_type = pd.Series(data.select_dtypes(include=['float', 'int']).columns)

imp_mode = SimpleImputer(missing_values = np.nan, strategy="most_frequent", copy=True)
test_cate = imp_mode.fit_transform(test_cate)

imp_median = SimpleImputer(missing_values = np.nan, strategy="median", copy=True)
test_float = imp_median.fit_transform(test_float)

In [37]:
data[cate_type] = test_cate
data[float_type] = test_float

In [38]:
data.head()

,id,P_ID,LOG ID,1-1_D,1-1_M,1-1_cell,1-1_value,1-2_D,1-2_M,1-2_cell,...,5-3_value,6-1_D,6-1_M,6-1_cell,6-1_value,6-2_cell,6-2_value,6-3_cell,6-3_value,TYPE
0,2.0,1542.0,7.0,主材,M1,1.0,27.382609,主材,M1,1.0,...,7.072,主材,M9,1.0,18.83,1.0,0.0,1.0,0.0,TYPE6
1,4.0,2804.0,9.0,主材,M1,1.0,26.108696,主材,M1,1.0,...,7.040,主材,M9,1.0,19.60,1.0,0.0,1.0,0.0,TYPE6
2,5.0,3463.0,10.0,主材,M1,1.0,27.330435,主材,M1,1.0,...,7.168,主材,M9,1.0,18.72,1.0,0.0,1.0,0.0,TYPE6
3,7.0,4873.0,12.0,主材,M1,1.0,26.460870,主材,M1,1.0,...,7.168,主材,M9,1.0,18.43,1.0,0.0,1.0,0.0,TYPE6
4,10.0,8155.0,16.0,主材,M1,1.0,26.365217,主材,M1,1.0,...,6.720,主材,M9,1.0,19.46,1.0,0.0,1.0,0.0,TYPE6


In [39]:
#if nonull that means 0
#if we sum up all values
#get 0, that means no null
su = 0
for i in data.isnull().sum():
    su += i

print(su)

0


In [40]:
data = pd.get_dummies(data)
print(data.head())

#after onehot encoding, let us check the columns
for i in data.columns:
    print(i, end = "")
    print(',', end = '')

     id    P_ID  LOG ID  1-1_cell  1-1_value  1-2_cell  1-2_value  1-3_cell  \
0   2.0  1542.0     7.0       1.0  27.382609       1.0  25.865217       1.0   
1   4.0  2804.0     9.0       1.0  26.108696       1.0  24.826087       1.0   
2   5.0  3463.0    10.0       1.0  27.330435       1.0  26.017391       1.0   
3   7.0  4873.0    12.0       1.0  26.460870       1.0  25.095652       1.0   
4  10.0  8155.0    16.0       1.0  26.365217       1.0  25.008696       1.0   

   1-3_value  2-1_cell  ...  6-1_D_主材  6-1_M_M23  6-1_M_M9  TYPE_TYPE1  \
0  13.960870       1.0  ...         1          0         1           0   
1  13.765217       1.0  ...         1          0         1           0   
2  14.439130       1.0  ...         1          0         1           0   
3  13.695652       1.0  ...         1          0         1           0   
4  13.652174       1.0  ...         1          0         1           0   

   TYPE_TYPE2  TYPE_TYPE3  TYPE_TYPE4  TYPE_TYPE5  TYPE_TYPE6  TYPE_TYPE7  
0   

In [41]:
all_id = data['id']
submission = pd.DataFrame(all_id, dtype='int')

In [42]:
#after we dealed with the fatures, standardlizing all features
# from sklearn.preprocessing import StandardScaler

# scale = StandardScaler() #z-scaler物件
# data = pd.DataFrame(scale.fit_transform(data),
#                                 columns=data.keys())

from sklearn.preprocessing import StandardScaler

scale = StandardScaler() #z-scaler物件
data[float_type] = pd.DataFrame(scale.fit_transform(data[float_type]),
                                columns=data[float_type].keys())

In [43]:
#save data
joblib.dump(data, 'data/X_test.pkl')

['data/X_test.pkl']

In [44]:
X_test = joblib.load('data/X_test.pkl')

In [45]:
X_test

,id,P_ID,LOG ID,1-1_cell,1-1_value,1-2_cell,1-2_value,1-3_cell,1-3_value,2-1_cell,...,6-1_D_主材,6-1_M_M23,6-1_M_M9,TYPE_TYPE1,TYPE_TYPE2,TYPE_TYPE3,TYPE_TYPE4,TYPE_TYPE5,TYPE_TYPE6,TYPE_TYPE7
0,-1.669545,-1.383459,-1.434227,-1.521984,0.260591,-1.521984,-0.299932,-0.206472,0.068556,-1.403246,...,1,0,1,0,0,0,0,0,1,0
1,-1.668894,-1.382983,-1.408165,-1.521984,-0.156679,-1.521984,-0.618820,-0.206472,-0.011832,-1.403246,...,1,0,1,0,0,0,0,0,1,0
2,-1.668569,-1.382734,-1.395134,-1.521984,0.243501,-1.521984,-0.253233,-0.206472,0.265060,-1.403246,...,1,0,1,0,0,0,0,0,1,0
3,-1.667919,-1.382202,-1.369072,-1.521984,-0.041325,-1.521984,-0.536096,-0.206472,-0.040415,-1.403246,...,1,0,1,0,0,0,0,0,1,0
4,-1.666944,-1.380963,-1.316948,-1.521984,-0.072656,-1.521984,-0.562781,-0.206472,-0.058279,-1.403246,...,1,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5718,1.800268,1.312158,-0.756613,0.429251,-0.164512,0.429251,-0.059098,-0.206472,-0.152959,0.460476,...,1,0,1,1,0,0,0,0,0,0
5719,1.800593,1.312270,-0.743582,0.429251,-0.164512,0.429251,-0.059098,-0.206472,-0.152959,0.460476,...,1,0,1,1,0,0,0,0,0,0
5720,1.800919,1.312362,-0.730551,0.429251,-0.164512,0.429251,-0.059098,-0.206472,-0.152959,0.460476,...,1,0,1,1,0,0,0,0,0,0
5721,1.802219,1.312671,-0.678427,0.429251,-0.164512,0.429251,-0.059098,-0.206472,-0.152959,0.460476,...,1,0,1,1,0,0,0,0,0,0


## Predict CIEX stacking1

In [62]:
model_type = ['RandomForest', 'SVR', 'ridge']
which_label = 'CIEX'


def predict_stacking1(X_test, model_type, which_label):
    X_meta_test = np.empty([5723, 0])
    for i in range(3):
        tm = np.zeros([5723, 1])
        for j in range(3):
            R_feat = joblib.load(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
#             print(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
            X_test_R = X_test[R_feat]
            model = joblib.load(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            print(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            pred = model.predict(X_test_R).reshape(-1, 1)
            tm += pred
        tm /= 3
        X_meta_test = np.hstack((X_meta_test , tm))
        print(X_meta_test.shape)
    return X_meta_test

X_meta_test = predict_stacking1(X_test, model_type, which_label)

saved_model_CIEX/model0_RandomForest_stacking1.pkl
saved_model_CIEX/model1_RandomForest_stacking1.pkl
saved_model_CIEX/model2_RandomForest_stacking1.pkl
(5723, 1)
saved_model_CIEX/model0_SVR_stacking1.pkl
saved_model_CIEX/model1_SVR_stacking1.pkl
saved_model_CIEX/model2_SVR_stacking1.pkl
(5723, 2)
saved_model_CIEX/model0_ridge_stacking1.pkl
saved_model_CIEX/model1_ridge_stacking1.pkl
saved_model_CIEX/model2_ridge_stacking1.pkl
(5723, 3)


## Predict CIEX stacking2

In [63]:

def predict_stacking2(which_label):
    model = joblib.load(f'saved_model_{which_label}/RandomForest_stacking2.pkl')
    print(f'saved_model_{which_label}/RandomForest_stacking2.pkl')
    y_hat = model.predict(X_meta_test)
    return y_hat

#predict CIEX
pred = predict_stacking2(which_label)
print(pred)
submission[which_label] = pred

saved_model_CIEX/RandomForest_stacking2.pkl
[3.14236789 3.14236789 3.14236789 ... 0.89464033 0.89464033 0.89508853]


In [53]:
submission.head()

,id,CIEX,CIEY
0,2,3.142368,2.156949
1,4,3.142368,2.156949
2,5,3.142368,2.157060
3,7,3.142368,2.156949
4,10,3.142368,2.156949


## Predict CIEY stacking1

In [64]:
model_type = ['RandomForest', 'SVR', 'ridge']
which_label = 'CIEY'


def predict_stacking1(X_test, model_type, which_label):
    X_meta_test = np.empty([5723, 0])
    for i in range(3):
        tm = np.zeros([5723, 1])
        for j in range(3):
            R_feat = joblib.load(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
#             print(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
            X_test_R = X_test[R_feat]
            model = joblib.load(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            print(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            pred = model.predict(X_test_R).reshape(-1, 1)
            tm += pred
        tm /= 3
        X_meta_test = np.hstack((X_meta_test , tm))
        print(X_meta_test.shape)
    return X_meta_test

X_meta_test = predict_stacking1(X_test, model_type, which_label)

saved_model_CIEY/model0_RandomForest_stacking1.pkl
saved_model_CIEY/model1_RandomForest_stacking1.pkl
saved_model_CIEY/model2_RandomForest_stacking1.pkl
(5723, 1)
saved_model_CIEY/model0_SVR_stacking1.pkl
saved_model_CIEY/model1_SVR_stacking1.pkl
saved_model_CIEY/model2_SVR_stacking1.pkl
(5723, 2)
saved_model_CIEY/model0_ridge_stacking1.pkl
saved_model_CIEY/model1_ridge_stacking1.pkl
saved_model_CIEY/model2_ridge_stacking1.pkl
(5723, 3)


## Predict CIEY stacking2

In [65]:

def predict_stacking2(which_label):
    model = joblib.load(f'saved_model_{which_label}/RandomForest_stacking2.pkl')
    y_hat = model.predict(X_meta_test)
    return y_hat

#predict CIEY
pred = predict_stacking2(which_label)

submission[which_label] = pred

In [66]:
submission[which_label]

0       2.156949
1       2.156949
2       2.157060
3       2.156949
4       2.156949
          ...   
5718    4.563433
5719    4.564276
5720    4.564251
5721    4.564251
5722    4.564235
Name: CIEY, Length: 5723, dtype: float64

## Predict CIEX_DIFF stacking1

In [67]:
model_type = ['RandomForest', 'SVR', 'ridge']
which_label = 'CIEX_DIFF'


def predict_stacking1(X_test, model_type, which_label):
    X_meta_test = np.empty([5723, 0])
    for i in range(3):
        tm = np.zeros([5723, 1])
        for j in range(3):
            R_feat = joblib.load(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
#             print(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
            X_test_R = X_test[R_feat]
            model = joblib.load(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            print(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            pred = model.predict(X_test_R).reshape(-1, 1)
            tm += pred
        tm /= 3
        X_meta_test = np.hstack((X_meta_test , tm))
        print(X_meta_test.shape)
    return X_meta_test

X_meta_test = predict_stacking1(X_test, model_type, which_label)

saved_model_CIEX_DIFF/model0_RandomForest_stacking1.pkl
saved_model_CIEX_DIFF/model1_RandomForest_stacking1.pkl
saved_model_CIEX_DIFF/model2_RandomForest_stacking1.pkl
(5723, 1)
saved_model_CIEX_DIFF/model0_SVR_stacking1.pkl
saved_model_CIEX_DIFF/model1_SVR_stacking1.pkl
saved_model_CIEX_DIFF/model2_SVR_stacking1.pkl
(5723, 2)
saved_model_CIEX_DIFF/model0_ridge_stacking1.pkl
saved_model_CIEX_DIFF/model1_ridge_stacking1.pkl
saved_model_CIEX_DIFF/model2_ridge_stacking1.pkl
(5723, 3)


## Predict CIEX_DIFF stacking2

In [68]:

def predict_stacking2(which_label):
    model = joblib.load(f'saved_model_{which_label}/RandomForest_stacking2.pkl')
    y_hat = model.predict(X_meta_test)
    return y_hat

#predict
pred = predict_stacking2(which_label)

submission[which_label] = pred

## Predict CIEY_DIFF stacking1

In [69]:
model_type = ['RandomForest', 'SVR', 'ridge']
which_label = 'CIEY_DIFF'


def predict_stacking1(X_test, model_type, which_label):
    X_meta_test = np.empty([5723, 0])
    for i in range(3):
        tm = np.zeros([5723, 1])
        for j in range(3):
            R_feat = joblib.load(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
#             print(f'saved_model_{which_label}/feat_columns_{model_type[i]}_stacking1.pkl')
            X_test_R = X_test[R_feat]
            model = joblib.load(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            print(f'saved_model_{which_label}/model{j}_{model_type[i]}_stacking1.pkl')
            pred = model.predict(X_test_R).reshape(-1, 1)
            tm += pred
        tm /= 3
        X_meta_test = np.hstack((X_meta_test , tm))
        print(X_meta_test.shape)
    return X_meta_test

X_meta_test = predict_stacking1(X_test, model_type, which_label)

saved_model_CIEY_DIFF/model0_RandomForest_stacking1.pkl
saved_model_CIEY_DIFF/model1_RandomForest_stacking1.pkl
saved_model_CIEY_DIFF/model2_RandomForest_stacking1.pkl
(5723, 1)
saved_model_CIEY_DIFF/model0_SVR_stacking1.pkl
saved_model_CIEY_DIFF/model1_SVR_stacking1.pkl
saved_model_CIEY_DIFF/model2_SVR_stacking1.pkl
(5723, 2)
saved_model_CIEY_DIFF/model0_ridge_stacking1.pkl
saved_model_CIEY_DIFF/model1_ridge_stacking1.pkl
saved_model_CIEY_DIFF/model2_ridge_stacking1.pkl
(5723, 3)


## Predict CIEY_DIFF stacking2

In [70]:

def predict_stacking2(which_label):
    model = joblib.load(f'saved_model_{which_label}/RandomForest_stacking2.pkl')
    y_hat = model.predict(X_meta_test)
    return y_hat

#predict
pred = predict_stacking2(which_label)

submission[which_label] = pred

In [71]:
submission[which_label]

0       0.003389
1       0.004092
2       0.003495
3       0.003459
4       0.005081
          ...   
5718    0.002216
5719    0.002622
5720    0.002482
5721    0.002574
5722    0.002284
Name: CIEY_DIFF, Length: 5723, dtype: float64

In [ ]:
jiji

In [59]:
submission

,id,CIEX,CIEY,CIEX_DIFF,CIEY_DIFF
0,2,3.142368,2.156949,0.001808,0.003389
1,4,3.142368,2.156949,0.001256,0.004092
2,5,3.142368,2.157060,0.001256,0.003495
3,7,3.142368,2.156949,0.001256,0.003459
4,10,3.142368,2.156949,0.002201,0.005081
...,...,...,...,...,...
5718,10674,0.895584,4.563433,0.003106,0.002216
5719,10675,0.894640,4.564276,0.007043,0.002622
5720,10676,0.894640,4.564251,0.003486,0.002482
5721,10680,0.894640,4.564251,0.003486,0.002574


In [27]:
jij

NameError: name 'jij' is not defined

In [60]:
# Our final prediction!!!!!!!!!!!
submission.to_csv('data/sub2.csv', index=False)